<a href="https://colab.research.google.com/github/ACM-Research/NLP-Summarizer/blob/master/Notebooks/t5-summarizer/t5_tifu_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import os,sys
import json
import spacy
from collections import Counter


#T5 model Types
#
#t5-small
#t5-base
#t5-large
#t5-3B
#t5-11B

model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

#load Spacy
nlp = spacy.load("en_core_web_sm")

# Configures model to use GPU
device = torch.device('cuda')
model.to(torch.device("cuda:0"))

print('download complete')


download complete


In [5]:
path = "/content/drive/My Drive/tifu_all_tokenized_and_filtered.json"

In [ ]:
os.mkdir("/content/drive/My Drive/tifuSummaries")

In [9]:
data = []
with open(path) as file:
  for line in file:
    data.append(json.loads(line))

In [ ]:
  summaries = open("/content/drive/My Drive/tifuSummaries/tifu_summaries.json","w")

  for index,tifu in enumerate(data):
    if(index <= 100):
      jsonStuff = {}
      
      text = tifu["selftext_without_tldr"]

    # Adjust text stripper to make sure text is stripped properly
      preprocess_text = text.strip().replace("\n"," ")
      t5_prepared_Text = "summarize: "+preprocess_text
      print ("original text preprocessed: \n", preprocess_text)

      # Adjust max_length for speech size based on tokens
      tokenized_text = tokenizer.encode(t5_prepared_Text,truncation=True, return_tensors="pt").to(device)


      # summmarize 
      # Change no_repeat_ngram_size to adjust size of Ngrams in summary
      summary_ids = model.generate(tokenized_text,
                                          num_beams=4,
                                          no_repeat_ngram_size=8,
                                          min_length=30,
                                          max_length=100,
                                          early_stopping=True)

      output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

      print ("\nSummarized text: \n",output)
      print()

      original_text_pos = nlp(preprocess_text)
      summary_pos = nlp(output)

      print()

      jsonStuff = {"original_text:": preprocess_text, 
                  "summary_text": output,
                  "compression_ratio": str(len(output)/len(preprocess_text)),
                  "original_pos_tags": Counter([token.pos_ for token in original_text_pos]),
                  "summary_pos_tags": Counter([token.pos_ for token in summary_pos])
                   }
      
      summaries.write(json.dumps(jsonStuff) + "\n")
      print("written\n")

    else:
      break

  summaries.close()